<a href="https://colab.research.google.com/github/selychan/LstmModelling/blob/main/LSTM_Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [162]:
import pandas as pd

In [163]:
import csv
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [164]:
# prompt: tensorflow kütüphanesini ekle

import tensorflow as tf


In [165]:
import keras
from keras.models import Sequential
from keras.layers import LSTM,Dropout,Dense
from sklearn.preprocessing import LabelEncoder

In [166]:
test_data = pd.read_csv('/content/labelled_testing_data.csv', sep=';', quoting=csv.QUOTE_NONE)
test_data #test datasını hazırladık

,timestamp,processId,threadId,parentProcessId,userId,mountNamespace,processName,hostName,eventId,eventName,stackAddresses,argsNum,returnValue,args,sus,evil
0,129.050.634,382,382,1,101,4026532232,systemd-resolve,ip-10-100-1-217,41,socket,"[140159195621643, 140159192455417, 94656731598...",3,15,"[{'name': 'domain', 'type': 'int', 'value': 'A...",0.0,0.0
1,129.051.238,379,379,1,100,4026532231,systemd-network,ip-10-100-1-217,41,socket,"[139853228042507, 93935071185801, 93935080775184]",3,15,"[{'name': 'domain', 'type': 'int', 'value': 'A...",0.0,0.0
2,129.051.434,1,1,0,0,4026531840,systemd,ip-10-100-1-217,1005,security_file_open,"[140362867191588, 8103505641674583858]",4,0,"[{'name': 'pathname', 'type': 'const char*', '...",0.0,0.0
3,129.051.481,1,1,0,0,4026531840,systemd,ip-10-100-1-217,257,openat,[],4,17,"[{'name': 'dirfd', 'type': 'int', 'value': -10...",0.0,0.0
4,129.051.522,1,1,0,0,4026531840,systemd,ip-10-100-1-217,5,fstat,[140362867189385],2,0,"[{'name': 'fd', 'type': 'int', 'value': 17}, {...",0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188962,496.629.003,7555,7555,7548,1001,4026531840,tsm,ip-10-100-1-217,42,connect,[],3,-114,"""[{'name': 'sockfd', 'type': 'int', 'value': 4...",1.0,1.0
188963,496.629.025,7555,7555,7548,1001,4026531840,tsm,ip-10-100-1-217,42,connect,[],3,-114,"""[{'name': 'sockfd', 'type': 'int', 'value': 4...",1.0,1.0
188964,496.629.047,7555,7555,7548,1001,4026531840,tsm,ip-10-100-1-217,42,connect,[],3,-114,"""[{'name': 'sockfd', 'type': 'int', 'value': 4...",1.0,1.0
188965,496.629.069,7555,7555,7548,1001,4026531840,tsm,ip-10-100-1-217,42,connect,[],3,-114,"""[{'name': 'sockfd', 'type': 'int', 'value': 4...",1.0,1.0


In [167]:
label_encoder = LabelEncoder()

In [168]:
test_data['hostname_encoded'] = label_encoder.fit_transform(test_data['hostName'])


In [169]:
test_data['timestamp_encoded'] = label_encoder.fit_transform(test_data['timestamp'])


In [170]:
test_data.drop('hostName', axis=1, inplace=True)


In [171]:
test_data.drop('timestamp', axis=1, inplace=True)

In [173]:
test_data.drop('mountNamespace', axis=1, inplace=True)

In [172]:
test_data = test_data.drop([ 'processName', 'eventName', 'stackAddresses', 'args'], axis=1)

In [174]:
test_data.drop('hostName', axis=1, inplace=True)

KeyError: "['hostName'] not found in axis"

In [175]:
test_data

,processId,threadId,parentProcessId,userId,eventId,argsNum,returnValue,sus,evil,hostname_encoded,timestamp_encoded
0,382,382,1,101,41,3,15,0.0,0.0,0,37
1,379,379,1,100,41,3,15,0.0,0.0,0,38
2,1,1,0,0,1005,4,0,0.0,0.0,0,39
3,1,1,0,0,257,4,17,0.0,0.0,0,40
4,1,1,0,0,5,2,0,0.0,0.0,0,41
...,...,...,...,...,...,...,...,...,...,...,...
188962,7555,7555,7548,1001,42,3,-114,1.0,1.0,0,188962
188963,7555,7555,7548,1001,42,3,-114,1.0,1.0,0,188963
188964,7555,7555,7548,1001,42,3,-114,1.0,1.0,0,188964
188965,7555,7555,7548,1001,42,3,-114,1.0,1.0,0,188965


In [176]:
train_data = pd.read_csv('/content/labelled_training_data.csv', sep=';', quoting=csv.QUOTE_NONE)
train_data
#train datasını hazırladık.

,timestamp,processId,threadId,parentProcessId,userId,mountNamespace,processName,hostName,eventId,eventName,stackAddresses,argsNum,returnValue,args,sus,evil
0,1.809.495.787,381,7337,1,100,4026532231,close,ip-10-100-1-120,157,prctl,"[140662171848350, 11649800180280676]",5,0,"[{'name': 'option', 'type': 'int', 'value': 'P...",1,0
1,1.809.495.832,381,7337,1,100,4026532231,close,ip-10-100-1-120,3,close,[140662171777451],1,0,"[{'name': 'fd', 'type': 'int', 'value': 19}]",1,0
2,1.809.495.921,381,7337,1,100,4026532231,close,ip-10-100-1-120,1010,sched_process_exit,[],0,0,[],1,0
3,1.894.139.651,7347,7347,7341,0,4026531840,sh,ip-10-100-1-120,21,access,[],2,-2,"[{'name': 'pathname', 'type': 'const char*', '...",1,0
4,1.894.142.127,7347,7347,7341,0,4026531840,sh,ip-10-100-1-120,1005,security_file_open,"[139778263990104, 139778263906698]",4,0,"[{'name': 'pathname', 'type': 'const char*', '...",1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
763139,324.542.553,7274,7274,7108,0,4026531840,sshd,ip-10-100-1-79,1003,cap_capable,[],1,0,"[{'name': 'cap', 'type': 'int', 'value': 'CAP_...",0,0
763140,324.542.579,7274,7274,7108,0,4026531840,sshd,ip-10-100-1-79,62,kill,[],2,0,"[{'name': 'pid', 'type': 'pid_t', 'value': 0},...",0,0
763141,324.543.178,7274,7274,7108,0,4026531840,sshd,ip-10-100-1-79,1010,sched_process_exit,[],0,0,[],0,0
763142,324.543.695,7275,7275,1,109,4026531840,sshd,ip-10-100-1-79,1010,sched_process_exit,[],0,0,[],0,0


In [177]:
train_data['hostname_encoded'] = label_encoder.fit_transform(train_data['hostName'])
train_data.drop('hostName', axis=1, inplace=True)

In [178]:
train_data['timestamp_encoded'] = label_encoder.fit_transform(train_data['timestamp'])
train_data.drop('timestamp', axis=1, inplace=True)

In [179]:
train_data = train_data.drop([ 'processName', 'eventName', 'stackAddresses', 'mountNamespace','stackAddresses','args'], axis=1)

In [72]:
train_data

,processId,threadId,parentProcessId,userId,eventId,argsNum,returnValue,sus,evil,hostname_encoded,timestamp_encoded
0,381,7337,1,100,157,5,0,1,0,0,106827
1,381,7337,1,100,3,1,0,1,0,0,106828
2,381,7337,1,100,1010,0,0,1,0,0,106829
3,7347,7347,7341,0,21,2,-2,1,0,0,152954
4,7347,7347,7341,0,1005,4,0,1,0,0,152964
...,...,...,...,...,...,...,...,...,...,...,...
763139,7274,7274,7108,0,1003,1,0,0,0,6,567229
763140,7274,7274,7108,0,62,2,0,0,0,6,567230
763141,7274,7274,7108,0,1010,0,0,0,0,6,567231
763142,7275,7275,1,109,1010,0,0,0,0,6,567232


In [180]:
train_data = train_data.dropna()

In [181]:
test_data = test_data.dropna()

In [40]:
test_data

,processId,threadId,parentProcessId,userId,mountNamespace,eventId,argsNum,returnValue,sus,evil,hostname_encoded
0,382,382,1,101,4026532232,41,3,15,0.0,0.0,0
1,379,379,1,100,4026532231,41,3,15,0.0,0.0,0
2,1,1,0,0,4026531840,1005,4,0,0.0,0.0,0
3,1,1,0,0,4026531840,257,4,17,0.0,0.0,0
4,1,1,0,0,4026531840,5,2,0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...
50302,7555,7555,7548,1001,4026531840,42,3,-114,1.0,1.0,0
50303,7555,7555,7548,1001,4026531840,42,3,-114,1.0,1.0,0
50304,7555,7555,7548,1001,4026531840,42,3,-114,1.0,1.0,0
50305,7555,7555,7548,1001,4026531840,42,3,-114,1.0,1.0,0


In [182]:
X = test_data[['processId', 'eventId','hostname_encoded','argsNum','userId']] #2 adet bağımsız değişkenimiz mevcut
y = test_data[['evil']] #1 adet hedef değişkenimiz mevcut. sus ve evil.

# Veriyi train ve test setlerine bölelim test %20 eğitim %80
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [157]:
# prompt: f1 score hesaplar mısın

from sklearn.metrics import f1_score

f1 = f1_score(y_test, predictions)
print("f1 score:", f1)


f1 score: 1.0


In [136]:
import numpy as np

In [158]:


from sklearn.metrics import precision_score
precision = precision_score(y_test, predictions)
print("precision score:", precision)


precision score: 1.0


In [159]:


from sklearn.metrics import confusion_matrix

# Confusion matrix
cm = confusion_matrix(y_test, predictions)

print(cm)


[[ 6079     0]
 [    0 31715]]


In [183]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

# Veriyi train ve test setlerine bölelim test %20 eğitim %80
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = Sequential() #sıralı model oluşturduk.
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=5))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.compile(loss='mae', optimizer='adam')
model.fit(X_train, y_train['evil'], epochs=1, batch_size=32)
model.evaluate(X_test, y_test['evil'])


1182/1182 [==============================] - 5s 4ms/step - loss: 0.0508


0.05082233250141144

In [184]:
# prompt: accuracy hesapla

from sklearn.metrics import accuracy_score
y_pred = model.predict(X_test)
y_pred = np.round(y_pred)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


1182/1182 [==============================] - 8s 5ms/step
Accuracy: 0.9719267608615124
